In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from dotenv import load_dotenv 
import requests
import time
from bs4 import BeautifulSoup
from pathlib import Path
import re


IMAGES_DIR = Path("images")               # Folder to save images
SLIDESHOW_URL = "https://www.vogue.com/fashion-shows/spring-2026-ready-to-wear/christophe-lemaire/slideshow/collection#1"
WAIT_TIME = 15                            # Max wait time for elements to load
LAZY_LOAD_DELAY = 2                        # Time to wait for images to lazy-load
USER_AGENT = "Mozilla/5.0"     
IMAGES_DIR.mkdir(exist_ok=True)   

load_dotenv()

True

In [2]:
options = Options()
html = requests.get(SLIDESHOW_URL, headers={"User-Agent": USER_AGENT}).text
soup = BeautifulSoup(html, "html.parser")  
options.add_argument("--start-maximized")
driver = webdriver.Chrome()

driver.get("https://id.condenast.com/")
wait = WebDriverWait(driver, 15)

In [3]:
# The selector name may change when they update the website
vogue_login_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#main-content > div > div.utility-card__grid.utility-card > div:nth-child(16) > a"))
)
vogue_login_button.click()

email = os.getenv("VOGUE_EMAIL")
password = os.getenv("VOGUE_PASSWORD")

email_input = wait.until(
        EC.presence_of_element_located((By.NAME, "email"))
)
email_input.send_keys(email)

email_continue_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#email-continue-button"))
)
# Need to use execute_script as it uses javascript, simulating a real user click 
driver.execute_script("arguments[0].click();", email_continue_button)

password_input = wait.until(
        EC.presence_of_element_located((By.NAME, "password"))
)
password_input.send_keys(password)

sign_in_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#log-in-sign-in-button-password"))
)
driver.execute_script("arguments[0].click();", sign_in_button)


no_passkey_button = wait.until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#do-not-setup-passkey-button"))
)
driver.execute_script("arguments[0].click();", no_passkey_button)

# It's probbaly to do with cookies as to why it's not working, need to save the cookies somewhere when I switch websites
# driver.execute_script("window.location.href='https://www.vogue.com/fashion-shows/spring-2026-ready-to-wear/christophe-lemaire/slideshow/collection#1'")
driver.get(SLIDESHOW_URL)

In [4]:
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "img[data-src]"))
)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")


In [ ]:
def extract_image(slide_number: int):
    # Extract all images with a data-src
    img_tags = soup.find_all("img", {"data-src": True})

    if img_tags:
        img_url = img_tags[0]["data-src"]
        print("Image URL:", img_url)

        # Download the image
        headers = {"User-Agent": "Mozilla/5.0"}  # Pretend to be a browser
        img_data = requests.get(img_url, headers=headers).content

        filename = f"vogue_image_{slide_number}.jpg"
        filepath = os.path.join("images", filename)

        with open(filepath, "wb") as f:
            f.write(img_data)

        print("Image saved")
    else:
        print("No image found")

In [6]:
previous_slide = 0
while True: 

    current_url = driver.current_url
    match = re.search(r'#(\d+)', current_url)
    if not match:
        break
    
    current_slide = int(match.group(1))
    
    extract_image(current_slide)
    
    # Check if the current slide is lower than the previous slide to prevent duplicate images.
    if current_slide < previous_slide:
        break

    previous_slide = current_slide

    next_button = driver.find_element(By.CSS_SELECTOR, "div[aria-label='Next']")
    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
    next_button.click()
    time.sleep(2)  # wait for next image to load



Image URL: https://assets.vogue.com/photos/685c4877afc82a7e943bc182/master/w_2560%2Cc_limit/00001-lemaire-spring-2026-ready-to-wear-credit-gregoire-avenel.jpg
✅ Image saved as vogue_image.jpg
Image URL: https://assets.vogue.com/photos/685c4877afc82a7e943bc182/master/w_2560%2Cc_limit/00001-lemaire-spring-2026-ready-to-wear-credit-gregoire-avenel.jpg
✅ Image saved as vogue_image.jpg
Image URL: https://assets.vogue.com/photos/685c4877afc82a7e943bc182/master/w_2560%2Cc_limit/00001-lemaire-spring-2026-ready-to-wear-credit-gregoire-avenel.jpg
✅ Image saved as vogue_image.jpg
Image URL: https://assets.vogue.com/photos/685c4877afc82a7e943bc182/master/w_2560%2Cc_limit/00001-lemaire-spring-2026-ready-to-wear-credit-gregoire-avenel.jpg
✅ Image saved as vogue_image.jpg
Image URL: https://assets.vogue.com/photos/685c4877afc82a7e943bc182/master/w_2560%2Cc_limit/00001-lemaire-spring-2026-ready-to-wear-credit-gregoire-avenel.jpg
✅ Image saved as vogue_image.jpg
Image URL: https://assets.vogue.com/phot